### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


In [21]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100%; !important; }</style>"))

**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [316]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [473]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self._imp = self._gini_imp
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self._imp = self._entropy_imp
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self._imp = self._misclass_imp
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def _gini_imp(self, y):
        un, counts = np.unique(y, return_counts=True)
        return (1 - np.sum((counts / y.size) ** 2))*y.size

    def _entropy_imp(self, y):
        un, counts = np.unique(y, return_counts=True)
        return - np.sum((counts / y.size) * np.log2(counts / y.size))*y.size

    def _misclass_imp(self, y):
        un, counts = np.unique(y, return_counts=True)
        return (1 - np.max(counts / y.size))*y.size

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - (((l_c / l_s)**2).sum(axis=1))).
                reshape(-1, 1) * l_s +
                (1 - (((r_c / r_s)**2).sum(axis=1))).
                reshape(-1, 1) * r_s) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(((l_c/l_s) * np.log2(l_c/l_s)).sum(axis=1).
                 reshape(-1, 1) * l_s +
                 ((r_c/r_s) * np.log2(r_c/r_s)).sum(axis=1).
                 reshape(-1, 1) * r_s) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return ((1 - (l_c/l_s).max(axis=1)).reshape(-1, 1) * l_s +
                (1 - (r_c/r_s).max(axis=1)).reshape(-1, 1) * r_s) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_s, y_s = self.__sort_samples(x, y)
        borders = np.where(y_s[1:] != y_s[:-1])[0] + 1

        if len(borders) == 0:
            return np.inf, None

        counts = borders - np.append([0], borders[:-1])
        template = np.zeros((borders.shape[0], self.num_class))
        template[np.arange(borders.shape[0]), y_s[borders - 1]] = 1
        matrix_counts = template * counts.reshape(-1, 1)

        l_c = np.cumsum(matrix_counts, axis=0)
        r_c = np.bincount(y_s, minlength=self.num_class) - l_c
        l_s = borders.reshape(l_c.shape[0], 1)
        r_s = y_s.shape[0] - l_s

        gain = self.G_function(l_c, l_s, r_c, r_s)
        indx = np.argmin(gain)
        trh_ind = l_s[indx][0]

        return gain[indx], 0.5*(x_s[trh_ind - 1] + x_s[trh_ind])

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and depth >= self.max_depth) or \
                (y.size < self.min_samples_split) or \
                (np.bincount(y).argmax() / y.size >= self.sufficient_share):
            self.tree[node_id] = (self.LEAF_TYPE,
                                  np.bincount(y).argmax(),
                                  np.bincount(y) / y.size)
            return
        else:
            feature_ids = self.get_feature_ids(x.shape[1])
            thresholds = np.array([self.__find_threshold(x[:, i], y)
                                   for i in feature_ids])
            best_ind = thresholds[:, 0].argmin()
            best_thr = thresholds[best_ind, 1]

            best_imp = thresholds[best_ind, 0]
            if (best_thr is None):
                self.tree[node_id] = (self.LEAF_TYPE,
                                      np.bincount(y).argmax(),
                                      np.bincount(y) / y.size)
                return

            x_l, x_r, y_l, y_r = self.__div_samples(x, y,
                                                    feature_ids[best_ind],
                                                    best_thr)

            if y_l.size == 0 or y_r.size == 0:
                self.tree[node_id] = (self.LEAF_TYPE,
                                      np.bincount(y).argmax(),
                                      np.bincount(y) / y.size)
                return

            self.tree[node_id] = (self.NON_LEAF_TYPE,
                                  feature_ids[best_ind],
                                  best_thr)

            self.feature_importances_[
                feature_ids[best_ind]
            ] += self._imp(y) - self._imp(y_r) - self._imp(y_l)
            self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1], dtype=np.float)
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [402]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [403]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [424]:
%time clf.fit(X_train, y_train)

CPU times: user 2.69 ms, sys: 0 ns, total: 2.69 ms
Wall time: 5.72 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [425]:
%time my_clf.fit(X_train, y_train)

CPU times: user 27.4 ms, sys: 294 µs, total: 27.7 ms
Wall time: 35.1 ms


## Проверка качества работы на wine

In [426]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

In [427]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [428]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='latin1')
df = df.iloc[:, :97]

df = df.drop(['id', 'idg', 'condtn', 'position',
              'positin1', 'order', 'partner',
              'age_o', 'race_o', 'pf_o_att', 'tuition',
              'pf_o_sin', 'pf_o_int', 'pf_o_fun',
              'pf_o_amb', 'pf_o_sha', 'dec_o',
              'attr_o', 'sinc_o', 'intel_o', 'mn_sat',
              'fun_o', 'amb_o', 'shar_o', 'like_o',
              'prob_o','met_o', 'field', 'undergra',
              'from', 'zipcode', 'income', 'sports',
              'tvsports','exercise','dining', 'round',
              'museums','art','hiking','gaming',
              'clubbing','reading','tv','theater',
              'movies','concerts','music','shopping',
              'yoga', 'expnum'], axis=1)

df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df.field_cd = df.field_cd.astype(np.int)

df.race = df.race.astype(np.int)

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)
df.career_c = df.career_c.astype(np.int)

df = df.dropna(subset=['imprelig', 'imprace', 'date'])

In [429]:
df.loc[:, 'temp_totalsum'] =\
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
               'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                'fun1_1', 'amb1_1', 'shar1_1']].T/
     df.loc[:, 'temp_totalsum'].T).T * 100

In [430]:
df.loc[:, 'temp_totalsum'] =\
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
               'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
           'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                'fun2_1', 'amb2_1', 'shar2_1']].T/
     df.loc[:, 'temp_totalsum'].T).T * 100

In [431]:
df = df.drop(['temp_totalsum'], axis=1)

In [432]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)

In [433]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr',
                                          'samerace'], axis=1).dropna()
        
df_female.columns = df_female.columns + '_f'

df_male.pid = df_male.pid.astype(np.int)
df_female.pid_f = df_female.pid_f.astype(np.int)

In [434]:
df_join = df_male.join(df_female.set_index('iid_f'), how='right', on='pid')

In [435]:
target = df_join.loc[:, 'match'].values
df_join = df_join.drop(['match', 'iid', 'pid', 'pid_f'], axis=1)
data = df_join.values

In [436]:
SDD_my_clf = MyDecisionTreeClassifier(min_samples_split=2)
SDD_clf = DecisionTreeClassifier(min_samples_split=2)

In [437]:
X_train_SDD, X_test_SDD, y_train_SDD, y_test_SDD = train_test_split(data, target,
                                                    test_size=0.01, stratify=target)

## Проверка скорости работы на Speed Dating Data 

In [438]:
%time SDD_clf.fit(X_train_SDD, y_train_SDD)

CPU times: user 120 ms, sys: 3.23 ms, total: 124 ms
Wall time: 126 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [439]:
%time SDD_my_clf.fit(X_train_SDD, y_train_SDD)

CPU times: user 1.4 s, sys: 0 ns, total: 1.4 s
Wall time: 1.46 s


## Проверка качества работы на Speed Dating Data

In [440]:
f1_score(y_pred=SDD_clf.predict(X_test_SDD), y_true=y_test_SDD, average='macro')

0.4029850746268657

In [441]:
f1_score(y_pred=SDD_my_clf.predict(X_test_SDD), y_true=y_test_SDD, average='macro')

0.4444444444444444

## Задание 3

## Задание 4

In [447]:
f_idx = np.argsort(SDD_clf.feature_importances_)[::-1]
my_f_idx = np.argsort(SDD_my_clf.feature_importances_)[::-1]

features = zip(df_join.columns[f_idx][:10], SDD_clf.feature_importances_[f_idx][:10])
my_features = zip(df_join.columns[my_f_idx][:10], SDD_clf.feature_importances_[my_f_idx][:10])

print('Sklearn DecisionTreeClassifier:\n', *features, sep='\n')
print('\nMyDecisionTreeClassifier:\n', *my_features, sep='\n')

Sklearn DecisionTreeClassifier:

('int_corr', 0.08554163321322455)
('age', 0.03332227945042851)
('intel1_1', 0.029294935321020392)
('amb3_1_f', 0.02781611438956904)
('shar1_1_f', 0.02766568903140461)
('fun2_1', 0.02604916028016846)
('intel1_1_f', 0.025906587377460225)
('sinc2_1_f', 0.025816263862292625)
('field_cd_f', 0.025763095702396648)
('shar2_1_f', 0.025165177380294355)

MyDecisionTreeClassifier:

('shar1_1', 0.00797441761375645)
('amb1_1_f', 0.01906896294144787)
('int_corr', 0.08554163321322455)
('shar2_1', 0.021566769617078374)
('attr3_1', 0.021407228111603723)
('shar1_1_f', 0.02766568903140461)
('amb1_1', 0.022686319108339927)
('imprace', 0.013953473697108776)
('age', 0.03332227945042851)
('go_out_f', 0.013536544014855371)


## Задание 5

In [446]:
param_grid = {
    "max_depth": range(2, 10),
    "min_samples_split": range(2, 10),
    "criterion": ["gini", "entropy"],
    "n_estimators": range(5, 100, 5)}

model = RandomForestClassifier()
random_search = RandomizedSearchCV(model, param_grid, cv=5, n_iter=400)
random_search.fit(X_train, y_train)
random_search.best_params_

/home/waff/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_estimators': 95,
 'min_samples_split': 6,
 'max_depth': 5,
 'criterion': 'entropy'}